In [1]:
import pandas as pd
import numpy as np
import re
from completing_missing_clinician_questionnaire_2021 import map_additional_column_name_to_2021_clinician_column_name, should_remove_from_stu_columns, map_additional_column_name_to_2021_student_column_name , fill_id, IsClinician

In [2]:

# consts
mini_start_2022 ='mini_kid_sum_clin_timestamp'
mini_end_2022 = 'mini_kid_sum_clin_complete'

mini_start_add = 'mini_kid_sum_timestamp'
mini_end_add = 'mini_kid_sum_complete'



clinician_start = 'opening_clinicians_complete'
clinician_end_2021_2022 = 'cssrs_t_clin_complete'#, 'cssrs_fw_maya_complete'
clinician_end_add = 'cssrs_t_complete'

student_start = 'opening_students_timestamp'
student_end = 'cssrs_t_stu_complete'


maya_start_2022 = 'cssrs_fw_maya_timestamp'
maya_end_2022 = 'cssrs_fw_maya_complete'


# Chameleon - only student


chameleon_start = 'chameleon_timestamp'

chameleon_end = 'chameleon_complete'

In [3]:
old_data = pd.read_csv(r"../../Data/OriginalDataset/Schneider Depression Clinic Database.csv", na_values= ' ')
redcap_data = pd.read_csv(r"../../Data/OriginalDataset/ImmiRiskIPT2022_DATA_2023-09-03_1503.csv", na_values= ' ')
imputation_data = pd.read_csv(r"../../Data/helper_docs/Student_Clinician_data_2021.csv", na_values= ' ')
# the warning rooted from the nan values


C:\Users\nogur\anaconda3\envs\DeppClinic\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\envs\DeppClinic\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,46,51,53,99,325,366,439,453,459,464,466,501,507,512,514,519,521,530,532,569,571,580,599,766,771,773,779,784,789,794,798,800,804,809,811,813,816,818,827,829,849,866,868,877,896,906,941,964,1002,1014,1025,1063,1068,1070,1076,1081,1086,1091,1093,1095,1097,1099,1100,1126,1142,1149,1165,1168,1172,1173,1177,1178,1179,1180,1182,1183,1184,1187,1188,1190,1207,1212,1229,1232,1260,1262,1264,1266,1268,1271,1281,1283,1285,1291,1293,1297,1305,1306,1319,1321,1324,1326,1338,1340,1342,1343,1344,1350,1351,1352,1353,1465,1467,1468,1470,1486,1489,1492,1496,1513,1515,1516,1517,1523,1524,1525,1526,1580,1650,1653,1654) have mixed types.Speci

In [4]:


rename_imputation_data = {
    'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp',
    'cssrs_followup_complete':'cssrs_fw_maya_complete',
}

rename_old_data = {
    'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp',
    'cssrs_followup_complete':'cssrs_fw_maya_complete',
    'chameleon_complete_stu': 'chameleon_complete'
}

## error record_id = 363
imputation_data = imputation_data[imputation_data['record_id']!=363]

#do the imputation id like of redcap dataset
imputation_data = fill_id(imputation_data)


imputation_data = imputation_data.rename(rename_imputation_data, axis=1)

old_data = old_data.rename(rename_old_data, axis=1)



In [5]:
old_data_column_names = list(old_data.columns)
redcap_data_column_names = list(redcap_data.columns)
imputation_data_column_names = list(imputation_data.columns)


In [6]:

### clinician columns
old_data_clinician_columns = old_data_column_names[old_data_column_names.index(clinician_start):old_data_column_names.index(clinician_end_2021_2022) + 1]


redcap_data_clinician_columns = redcap_data_column_names[redcap_data_column_names.index(clinician_start):redcap_data_column_names.index(clinician_end_2021_2022) + 1]

redcap_data_clinician_columns += redcap_data_column_names[redcap_data_column_names.index(maya_start_2022):redcap_data_column_names.index(maya_end_2022) + 1]

imputation_data_clinician_columns = imputation_data_column_names[imputation_data_column_names.index(clinician_start):imputation_data_column_names.index(clinician_end_add)+1]


In [7]:
### student columns

old_data_student_columns = old_data_column_names[old_data_column_names.index(student_start):old_data_column_names.index(student_end)+1]
redcap_data_student_columns = redcap_data_column_names[redcap_data_column_names.index(student_start):redcap_data_column_names.index(student_end)+1] 

#+ cols_2022[cols_2022.index(maya_start_2022):cols_2022.index(maya_end_2022)+1]


imputation_data_student_columns = imputation_data_column_names[imputation_data_column_names.index(clinician_start):imputation_data_column_names.index(clinician_end_add)+1]
imputation_data_student_columns = imputation_data_student_columns + ['who', 'who_other', 'name']

imputation_data_student_columns = [i for i in imputation_data_student_columns if not should_remove_from_stu_columns(i)]



In [8]:
# Chameleon columns

old_data_chameleon_columns = old_data_column_names[old_data_column_names.index(chameleon_start):old_data_column_names.index(chameleon_end)+1]
redcap_data_chameleon_columns = redcap_data_column_names[redcap_data_column_names.index(chameleon_start):redcap_data_column_names.index(chameleon_end)+1] 
imputation_data_chameleon_columns = imputation_data_column_names[imputation_data_column_names.index(chameleon_start):imputation_data_column_names.index(chameleon_end)+1]


In [9]:
map_chameleon_2021_add = {}

for chameleon_column in old_data_chameleon_columns:
    new_name, success = map_additional_column_name_to_2021_student_column_name(chameleon_column, redcap_data_chameleon_columns)
    if success:
        map_chameleon_2021_add[chameleon_column] = new_name
    else:
        print(chameleon_column)

In [10]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres password=asdf123")

In [11]:
cur = conn.cursor()

In [12]:
cur.execute("""UPDATE  auxiliary_questionnaires_data.questionnaires_columns_names
SET old_data_column_names = '{0}'
WHERE questionnaire_name = 'chameleon';""".format(repr(map_chameleon_2021_add).replace("'", "\"")))


In [67]:
repr(map_chameleon_2021_add).replace({'"':"'", "'":'"'})

TypeError: replace expected at least 2 arguments, got 1

In [54]:
1831-1797

34

In [33]:
map_for_student_data = {}

for student_column in old_data_student_columns:
    new_name, success = map_additional_column_name_to_2021_student_column_name(student_column,redcap_data_student_columns)
    if success:
        map_for_student_data[student_column] = new_name
    else:
        print(student_column)

In [34]:
map_for_student_data

{'opening_students_timestamp': 'opening_students_timestamp',
 'who_stu': 'who_stu',
 'who_other_stu': 'who_other_stu',
 'name_stu': 'name_stu',
 'opening_students_complete': 'opening_students_complete',
 'trqsfmaris_stu_timestamp': 'trqsfmaris_stu_timestamp',
 'trq_sf_maris_1_stu': 'trq_sf_maris_1_stu',
 'trq_sf_maris_2_stu': 'trq_sf_maris_2_stu',
 'trq_sf_maris_3_stu': 'trq_sf_maris_3_stu',
 'trq_sf_maris_4_stu': 'trq_sf_maris_4_stu',
 'trq_sf_maris_5_stu': 'trq_sf_maris_5_stu',
 'trq_sf_maris_6_stu': 'trq_sf_maris_6_stu',
 'trq_sf_maris_7_stu': 'trq_sf_maris_7_stu',
 'trq_sf_maris_8_stu': 'trq_sf_maris_8_stu',
 'trq_sf_maris_9_stu': 'trq_sf_maris_9_stu',
 'trq_sf_maris_10_stu': 'trq_sf_maris_10_stu',
 'trq_sf_maris_11_stu': 'trq_sf_maris_11_stu',
 'trq_sf_maris_12_stu': 'trq_sf_maris_12_stu',
 'trq_sf_maris_13_stu': 'trq_sf_maris_13_stu',
 'trq_sf_maris_14_stu': 'trq_sf_maris_14_stu',
 'trqsfmaris_stu_complete': 'trqsfmaris_stu_complete',
 'cgi_s_stu_timestamp': 'cgi_s_stu_timestamp'

In [47]:
map_for_clinician_data = {}

for clinician_column in old_data_clinician_columns:
    new_name, success = map_additional_column_name_to_2021_clinician_column_name(clinician_column, redcap_data_clinician_columns)
    if success:
        map_for_clinician_data[clinician_column] = new_name
     

c_ssrs_t_2weeks_1_clin


In [36]:
map_for_clinician_data

{'opening_clinicians_complete': 'opening_clinicians_complete',
 'wai_immirisk_clin_timestamp': 'wai_immirisk_clin_timestamp',
 'wai_t_immi_1_clin': 'wai_t_immi_1_clin',
 'wai_t_immi_2_clin': 'wai_t_immi_2_clin',
 'wai_t_immi_3_clin': 'wai_t_immi_3_clin',
 'wai_t_immi_4_clin': 'wai_t_immi_4_clin',
 'wai_t_immi_5_clin': 'wai_t_immi_5_clin',
 'wai_t_immi_6_clin': 'wai_t_immi_6_clin',
 'wai_t_immi_7_clin': 'wai_t_immi_7_clin',
 'wai_t_immi_8_clin': 'wai_t_immi_8_clin',
 'wai_t_immi_9_clin': 'wai_t_immi_9_clin',
 'wai_t_immi_10_clin': 'wai_t_immi_10_clin',
 'wai_t_immi_11_clin': 'wai_t_immi_11_clin',
 'wai_t_immi_12_clin': 'wai_t_immi_12_clin',
 'wai_immirisk_clin_complete': 'wai_immirisk_clin_complete',
 'marisyscars_clin_timestamp': 'marisyscars_clin_timestamp',
 'maris_y_scars_1_clin': 'maris_y_scars_1_clin',
 'maris_y_scars_2_clin': 'maris_y_scars_2_clin',
 'maris_y_scars_3_clin': 'maris_y_scars_3_clin',
 'maris_y_scars_4_clin': 'maris_y_scars_4_clin',
 'maris_y_scars_5_clin': 'maris_y_

In [51]:
[i in old_data_student_columns  for i in redcap_data_student_columns]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 